# The AI Clinical Trials Architect: A Self-Evolving Agent Guild for Multi-Objective RAG Optimization

## Part 0: The Grand Challenge: Automating Clinical Trial Design

### 0.1. The New Frontier: From Answering Questions to Generating Protocols

Welcome to a deep dive into the next generation of agentic systems. In previous explorations, we've seen AI systems that can answer questions or summarize documents. Today, we elevate our ambitions significantly. Our goal is to build an AI system that can tackle a core, high-value task in the biopharmaceutical industry: **drafting the "Patient Inclusion/Exclusion Criteria" for a new clinical trial.**

This task is orders of magnitude more complex than simple RAG. It involves synthesizing information from disparate sources (medical literature, regulatory law, population data), understanding complex trade-offs, and generating a structured, compliant, and feasible document. It is a perfect testbed for a truly intelligent, multi-agent system.

### 0.2. The Multi-Objective Optimization Problem

A successful clinical trial isn't just scientifically sound; it's a delicate balance of competing priorities. Our AI system must learn to navigate this complex, multi-dimensional trade-off space. We will not be optimizing for a single score, but for a **Pareto Frontier** across five pillars:

1.  **Scientific Rigor:** How well do the criteria isolate the target patient population based on the latest medical research?
2.  **Regulatory Compliance:** Do the criteria adhere to FDA guidelines and established legal precedents?
3.  **Ethical Soundness:** Are vulnerable populations appropriately protected? Is the trial design fair and just?
4.  **Recruitment Feasibility:** Can a hospital realistically find enough patients who meet these criteria based on real-world population data?
5.  **Operational Simplicity (Cost Proxy):** Are the screening procedures required by the criteria simple and inexpensive, or do they require complex, costly tests?

### 0.3. The Architectural Vision: A Hierarchical Agent-of-Agents System

To solve this, we will build a hierarchical system:

-   **The Inner Loop (The "Trial Design Guild"):** A collaborative team of specialist agents built with LangGraph. This guild is our advanced RAG pipeline. It takes a trial concept and drafts a single criteria document.
-   **The Outer Loop (The "AI Research Director"):** A supervisory agent system, also built with LangGraph. It acts as the manager, observing the Guild's performance across all five objectives. It then intelligently evolves the Guild's internal "Standard Operating Procedures" (SOPs) — its configurations, tools, and prompts — to continuously improve its performance.

### 0.4. Visual Blueprint

<!-- Image of the Hierarchical System Diagram will be placed here -->

This notebook will walk you through building this entire system from the ground up, one function at a time. Let's begin.

## Part 1: Assembling the Knowledge Arsenal: Data, Models, and Tools

First, we need to set up our environment. This includes installing all necessary libraries, configuring our local LLM server, and preparing our diverse data sources. Unlike a simple demonstration, we will be downloading and processing real-world data to ground our system in reality.

### 1.1. Installing the Open-Source Stack

This cell contains all the necessary Python libraries. We use `langchain` and `langgraph` for our agentic framework, `ollama` to interact with local LLMs, `duckdb` for structured data, and specialized libraries like `biopython` for PubMed access and `pypdf` for regulatory documents.

In [1]:
%pip install langchain langgraph langchain_community langchain_openai langchain_core ollama pandas duckdb faiss-cpu sentence-transformers biopython pypdf pydantic lxml html2text beautifulsoup4 matplotlib -qqq

Note: you may need to restart the kernel to use updated packages.


### 1.2. Environment Configuration & Imports

Next, we'll set up our environment variables and import the core modules we'll be using throughout the notebook. For this project to work, you must have a LangSmith account to trace and evaluate our complex agent interactions. 

**Action Required:**
1. Create a `.env` file in the same directory as this notebook.
2. Add your LangSmith API keys to the `.env` file in the following format:
```
LANGCHAIN_API_KEY="ls__..."
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_PROJECT="AI_Clinical_Trials_Architect"
ENTREZ_EMAIL="your.email@example.com" 
```
*Note: A valid email is required by NCBI for PubMed API access.*

In [2]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

# Check if the environment variables are set
if "LANGCHAIN_API_KEY" not in os.environ or "ENTREZ_EMAIL" not in os.environ:
    print("Required environment variables not set. Please set them in your .env file or environment.")
else:
    print("Environment variables loaded successfully.")

# Set project name
os.environ["LANGCHAIN_PROJECT"] = "AI_Clinical_Trials_Architect"
print(f"LangSmith tracing is configured for project '{os.environ['LANGCHAIN_PROJECT']}'.")

Environment variables loaded successfully.
LangSmith tracing is configured for project 'AI_Clinical_Trials_Architect'.


### 1.3. Configuring the Local LLM Foundry with Ollama

Our system relies on a suite of locally-hosted open-source models served via Ollama. This approach provides cost-effective, private, and powerful inference. We will define a dictionary to hold our model clients, making it easy to call different models for different tasks.

**Action Required:**
Before running the next cell, ensure Ollama is running and you have pulled the required models:
```bash
ollama pull llama3.1:8b-instruct
ollama pull qwen2:7b
ollama pull llama3:70b
ollama pull nomic-embed-text
```

In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

# Define our model suite
llm_config = {
    "planner": ChatOllama(model="llama3.1:8b-instruct", temperature=0.0, format='json'),
    "drafter": ChatOllama(model="qwen2:7b", temperature=0.2),
    "sql_coder": ChatOllama(model="qwen2:7b", temperature=0.0),
    "director": ChatOllama(model="llama3:70b", temperature=0.0, format='json'),
    "embedding_model": OllamaEmbeddings(model="nomic-embed-text")
}

print("LLM clients configured:")
print(f"Planner (llama3.1:8b-instruct): {llm_config['planner']}")
print(f"Drafter (qwen2:7b): {llm_config['drafter']}")
print(f"SQL Coder (qwen2:7b): {llm_config['sql_coder']}")
print(f"Director (llama3:70b): {llm_config['director']}")
print(f"Embedding Model (nomic-embed-text): {llm_config['embedding_model']}")

LLM clients configured:
Planner (llama3.1:8b-instruct): model='llama3.1:8b-instruct' temperature=0.0 format='json'
Drafter (qwen2:7b): model='qwen2:7b' temperature=0.2
SQL Coder (qwen2:7b): model='qwen2:7b' temperature=0.0
Director (llama3:70b): model='llama3:70b' temperature=0.0 format='json'
Embedding Model (nomic-embed-text): base_url='http://localhost:11434' model='nomic-embed-text' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None


C:\Users\admin\AppData\Local\Temp\ipykernel_11660\3409962068.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  "planner": ChatOllama(model="llama3.1:8b-instruct", temperature=0.0, format='json'),
C:\Users\admin\AppData\Local\Temp\ipykernel_11660\3409962068.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  "embedding_model": OllamaEmbeddings(model="nomic-embed-text")


### 1.4. Preparing the Knowledge Stores

Our agents need access to diverse, real-world information. We will create four distinct knowledge stores.

**Step 1.4.1: Create data directories.**

In [4]:
import os

data_paths = {
    "base": "./data",
    "pubmed": "./data/pubmed_articles",
    "fda": "./data/fda_guidelines",
    "ethics": "./data/ethical_guidelines",
    "mimic": "./data/mimic_db"
}

for path in data_paths.values():
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Created directory: {path}")

Created directory: ./data
Created directory: ./data/pubmed_articles
Created directory: ./data/fda_guidelines
Created directory: ./data/ethical_guidelines
Created directory: ./data/mimic_db


**Step 1.4.2: Fetch real PubMed abstracts.**

We use the `Bio.Entrez` library to connect to the NCBI PubMed database and download recent, relevant articles. This provides our `Medical Researcher` agent with up-to-date scientific literature.

In [5]:
from Bio import Entrez
from Bio import Medline

def download_pubmed_articles(query, max_articles=20):
    """Fetches abstracts from PubMed and saves them as text files."""
    Entrez.email = os.environ.get("ENTREZ_EMAIL")
    print(f"Fetching PubMed articles for query: {query}")
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_articles, sort="relevance")
    record = Entrez.read(handle)
    id_list = record["IdList"]
    print(f"Found {len(id_list)} article IDs.")
    
    print("Downloading articles...")
    handle = Entrez.efetch(db="pubmed", id=id_list, rettype="medline", retmode="text")
    records = Medline.parse(handle)
    
    count = 0
    for i, record in enumerate(records):
        pmid = record.get("PMID", "")
        title = record.get("TI", "No Title")
        abstract = record.get("AB", "No Abstract")
        if pmid:
            filepath = os.path.join(data_paths["pubmed"], f"{pmid}.txt")
            with open(filepath, "w") as f:
                f.write(f"Title: {title}\n\nAbstract: {abstract}")
            print(f"[{i+1}/{len(id_list)}] Fetching PMID: {pmid}... Saved to {filepath}")
            count += 1
    return count

pubmed_query = "(SGLT2 inhibitor) AND (type 2 diabetes) AND (renal impairment)"
num_downloaded = download_pubmed_articles(pubmed_query)
print(f"PubMed download complete. {num_downloaded} articles saved.")

Fetching PubMed articles for query: (SGLT2 inhibitor) AND (type 2 diabetes) AND (renal impairment)
Found 20 article IDs.
[1/20] Fetching PMID: 36945734... Saved to ./data/pubmed_articles\36945734.txt
[2/20] Fetching PMID: 40470996... Saved to ./data/pubmed_articles\40470996.txt
[3/20] Fetching PMID: 38914124... Saved to ./data/pubmed_articles\38914124.txt
[4/20] Fetching PMID: 30697905... Saved to ./data/pubmed_articles\30697905.txt
[5/20] Fetching PMID: 36335326... Saved to ./data/pubmed_articles\36335326.txt
[6/20] Fetching PMID: 36351458... Saved to ./data/pubmed_articles\36351458.txt
[7/20] Fetching PMID: 40327845... Saved to ./data/pubmed_articles\40327845.txt
[8/20] Fetching PMID: 35113333... Saved to ./data/pubmed_articles\35113333.txt
[9/20] Fetching PMID: 34619106... Saved to ./data/pubmed_articles\34619106.txt
[10/20] Fetching PMID: 33413348... Saved to ./data/pubmed_articles\33413348.txt
[11/20] Fetching PMID: 34272327... Saved to ./data/pubmed_articles\34272327.txt
[12/20] 

**Step 1.4.3: Download a real FDA guideline document.**

This represents the regulatory information our `Regulatory Specialist` agent will use. We download a relevant PDF guidance document directly from the FDA website.

In [6]:
import requests
from pypdf import PdfReader
import io

def download_and_extract_text_from_pdf(url, output_path):
    print(f"Downloading FDA Guideline: {url}")
    try:
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes
        
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded and saved to {output_path}")
        
        # Now extract text to a .txt file for our loader
        reader = PdfReader(io.BytesIO(response.content))
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n\n"
        
        txt_output_path = os.path.splitext(output_path)[0] + '.txt'
        with open(txt_output_path, 'w') as f:
            f.write(text)
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
        return False

# This is a real FDA guidance document for developing drugs for diabetes
fda_url = "https://www.fda.gov/media/71185/download"
fda_pdf_path = os.path.join(data_paths["fda"], "fda_diabetes_guidance.pdf")
download_and_extract_text_from_pdf(fda_url, fda_pdf_path)

Successfully downloaded and saved to ./data/fda_guidelines/fda_diabetes_guidance.pdf


**Step 1.4.4: Create a sample Clinical Trial Ethics document.**

For the `Ethics Specialist`, we will create a document summarizing key principles from the Belmont Report, a foundational text in human subject research ethics.

In [7]:
ethics_content = """
Title: Summary of the Belmont Report Principles for Clinical Research

1. Respect for Persons: This principle requires that individuals be treated as autonomous agents and that persons with diminished autonomy are entitled to protection. This translates to robust informed consent processes. Inclusion/exclusion criteria must not unduly target or coerce vulnerable populations, such as economically disadvantaged individuals, prisoners, or those with severe cognitive impairments, unless the research is directly intended to benefit that population.

2. Beneficence: This principle involves two complementary rules: (1) do not harm and (2) maximize possible benefits and minimize possible harms. The criteria must be designed to select a population that is most likely to benefit and least likely to be harmed by the intervention. The risks to subjects must be reasonable in relation to anticipated benefits.

3. Justice: This principle concerns the fairness of distribution of the burdens and benefits of research. The selection of research subjects must be equitable. Criteria should not be designed to exclude certain groups without a sound scientific or safety-related justification. For example, excluding participants based on race, gender, or socioeconomic status is unjust unless there is a clear rationale related to the drug's mechanism or risk profile.
"""

ethics_path = os.path.join(data_paths["ethics"], "belmont_summary.txt")
with open(ethics_path, "w") as f:
    f.write(ethics_content)

print(f"Created ethics guideline file: {ethics_path}")

Created ethics guideline file: ./data/ethical_guidelines/belmont_summary.txt


**Step 1.4.5: Download and Load the MIMIC-III Dataset**

To ground our system in reality, we will now use the actual MIMIC-III dataset. Access to this dataset is controlled and requires credentialing.

**Action Required:**
1.  **Get Access:** Go to the [MIMIC-III PhysioNet page](https://physionet.org/content/mimiciii/1.4/) and follow the instructions to become a credentialed user. This is a required step for ethical and legal access to the data.
2.  **Download from Kaggle:** Once you have access, you can download the dataset from official sources or from [MIMIC-III Dataset page](https://physionet.org/content/mimiciii/1.4/). You will need to download the following three files:
    -   `PATIENTS.csv.gz`
    -   `DIAGNOSES_ICD.csv.gz`
    -   `LABEVENTS.csv.gz`
3.  **Place the Files:** Create a directory `mimiciii_csvs` inside your `./data/mimic_db/` folder. Place the three downloaded `.csv.gz` files into this new directory. Your file structure should look like this:
    ```
    ./data/mimic_db/mimiciii_csvs/PATIENTS.csv.gz
    ./data/mimic_db/mimiciii_csvs/DIAGNOSES_ICD.csv.gz
    ./data/mimic_db/mimiciii_csvs/LABEVENTS.csv.gz
    ```
The next cell will check for these files, and if found, will load them into a highly efficient DuckDB database. **Note:** Loading `LABEVENTS.csv.gz` can take several minutes and consume significant RAM due to its size (>250 million rows).

In [8]:
import duckdb
import pandas as pd
import os

def load_real_mimic_data():
    """Loads real MIMIC-III CSVs into a DuckDB database."""
    print("Attempting to load real MIMIC-III data from local CSVs...")
    db_path = os.path.join(data_paths["mimic"], "mimic3_real.db")
    csv_dir = os.path.join(data_paths["mimic"], "mimiciii_csvs")
    
    required_files = {
        "patients": os.path.join(csv_dir, "PATIENTS.csv.gz"),
        "diagnoses": os.path.join(csv_dir, "DIAGNOSES_ICD.csv.gz"),
        "labevents": os.path.join(csv_dir, "LABEVENTS.csv.gz"),
    }
    
    missing_files = [path for path in required_files.values() if not os.path.exists(path)]
    if missing_files:
        print("ERROR: The following MIMIC-III files were not found:")
        for f in missing_files:
            print(f"- {f}")
        print("\nPlease download them as instructed and place them in the correct directory.")
        return None
    
    print("Required files found. Proceeding with database creation.")
    if os.path.exists(db_path):
        os.remove(db_path)
    con = duckdb.connect(db_path)
    
    print(f"Loading {required_files['patients']} into DuckDB...")
    con.execute(f"CREATE TABLE patients AS SELECT SUBJECT_ID, GENDER, DOB, DOD FROM read_csv_auto('{required_files['patients']}')")
    
    print(f"Loading {required_files['diagnoses']} into DuckDB...")
    con.execute(f"CREATE TABLE diagnoses_icd AS SELECT SUBJECT_ID, ICD9_CODE FROM read_csv_auto('{required_files['diagnoses']}')")
    
    print(f"Loading and processing {required_files['labevents']} (this may take several minutes)...")
    # Labevents is huge. We read it as all text to avoid type errors, filter for our specific numeric lab items, then convert.
    con.execute(f"""CREATE TABLE labevents_staging AS 
                   SELECT SUBJECT_ID, ITEMID, VALUENUM 
                   FROM read_csv_auto('{required_files['labevents']}', all_varchar=True) 
                   WHERE ITEMID IN ('50912', '50852') AND VALUENUM IS NOT NULL AND VALUENUM ~ '^[0-9]+(\\.[0-9]+)?$'
                """)
    con.execute("CREATE TABLE labevents AS SELECT SUBJECT_ID, CAST(ITEMID AS INTEGER) AS ITEMID, CAST(VALUENUM AS DOUBLE) AS VALUENUM FROM labevents_staging")
    con.execute("DROP TABLE labevents_staging")

    con.close()
    return db_path

db_path = load_real_mimic_data()

if db_path:
    print(f"Real MIMIC-III database created at: {db_path}")
    print("\nTesting database connection and schema...")
    con = duckdb.connect(db_path)
    print(f"Tables in DB: {con.execute('SHOW TABLES').df()['name'].tolist()}")
    print("\nSample of 'patients' table:")
    # Note: Column names are uppercase in the real CSVs
    print(con.execute("SELECT * FROM patients LIMIT 5").df())
    print("\nSample of 'diagnoses_icd' table:")
    print(con.execute("SELECT * FROM diagnoses_icd LIMIT 5").df())
    con.close()


Attempting to load real MIMIC-III data from local CSVs...
Required files found. Proceeding with database creation.
Loading PATIENTS.csv.gz into DuckDB...
Loading DIAGNOSES_ICD.csv.gz into DuckDB...
Loading and processing LABEVENTS.csv.gz (this may take several minutes)...
Real MIMIC-III database created at: ./data/mimic_db/mimic3_real.db

Testing database connection and schema...
Tables in DB: ['patients', 'diagnoses_icd', 'labevents']

Sample of 'patients' table:
   ROW_ID  SUBJECT_ID GENDER         DOB         DOD    DOD_HOSP    DOD_SSN EXPIRE_FLAG
0      238       250      F  2164-12-27  2198-02-18  2198-02-18 2198-02-18           1
1      239       251      M  2078-02-21         NaN         NaN        NaN           0
2      240       252      M  2049-06-06  2123-09-01  2123-09-01 2123-09-01           1
3      241       253      F  2081-11-26         NaN         NaN        NaN           0
4      242       254      F  2028-04-12         NaN         NaN        NaN           0

Sample 

**Step 1.4.6: Create Vector Stores for Unstructured Data**

Now, we'll process our unstructured text files (PubMed, FDA, and Ethics docs) and load them into FAISS vector stores. This enables efficient semantic search for our RAG agents.

In [9]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

def create_vector_store(folder_path: str, embedding_model, store_name: str):
    """Loads documents from a folder, splits them, and creates a FAISS vector store."""
    print(f"--- Creating {store_name} Vector Store ---")
    loader = DirectoryLoader(folder_path, glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
    documents = loader.load()
    
    if not documents:
        print(f"No documents found in {folder_path}")
        return None, 0, 0
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    
    print(f"Loaded {len(documents)} documents, split into {len(texts)} chunks.")
    print("Generating embeddings and indexing into FAISS... (This may take a moment)")
    db = FAISS.from_documents(texts, embedding_model)
    print(f"{store_name} Vector Store created successfully.")
    return db, len(documents), len(texts)

def create_retrievers(embedding_model):
    pubmed_db, _, _ = create_vector_store(data_paths["pubmed"], embedding_model, "PubMed")
    fda_db, _, _ = create_vector_store(data_paths["fda"], embedding_model, "FDA")
    ethics_db, _, _ = create_vector_store(data_paths["ethics"], embedding_model, "Ethics")
    
    return {
        "pubmed_retriever": pubmed_db.as_retriever(search_kwargs={"k": 3}),
        "fda_retriever": fda_db.as_retriever(search_kwargs={"k": 3}),
        "ethics_retriever": ethics_db.as_retriever(search_kwargs={"k": 2}),
        "mimic_db_path": db_path
    }


In [10]:
knowledge_stores = create_retrievers(llm_config["embedding_model"])

print("Knowledge stores and retrievers created successfully.")
for name, store in knowledge_stores.items():
    print(f"{name}: {store}")

--- Creating PubMed Vector Store ---


100%|██████████| 20/20 [00:00<00:00, 1102.77it/s]


Loaded 20 documents, split into 35 chunks.
Generating embeddings and indexing into FAISS... (This may take a moment)


Batches: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


PubMed Vector Store created successfully.
--- Creating FDA Vector Store ---


...

FDA Vector Store created successfully.
--- Creating Ethics Vector Store ---


100%|██████████| 1/1 [00:00<00:00, 143.20it/s]


Loaded 1 documents, split into 1 chunks.
Generating embeddings and indexing into FAISS... (This may take a moment)


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.62it/s]


Ethics Vector Store created successfully.
Knowledge stores and retrievers created successfully.
pubmed_retriever: VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16b080510>)
fda_retriever: VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16a624a90>)
ethics_retriever: VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16b06e9d0>)
mimic_db_path: ./data/mimic_db/mimic3_synthetic.db


With our data downloaded, processed, and indexed, and our LLMs configured, we can now begin constructing the first major component of our system: The Trial Design Guild.

## Part 2: The Inner Loop: The "Trial Design Guild" - A Collaborative Agent Team

This is where we build our advanced RAG pipeline. It's not a simple chain, but a multi-agent system where specialists collaborate to produce a comprehensive output. The entire Guild's behavior is controlled by a single configuration object we call the `GuildSOP`.

### 2.1. Defining the "Guild's SOP" (The RAG Genome)

We use a Pydantic model to define the Standard Operating Procedures. This provides type safety and a clear, structured way to configure the Guild. This `GuildSOP` is the "genome" that our Outer Loop will evolve. We've expanded it to include our new Ethics specialist and more configuration options.

In [11]:
from pydantic import BaseModel, Field
from typing import Literal

class GuildSOP(BaseModel):
    """Standard Operating Procedures for the Trial Design Guild."""
    planner_prompt: str = Field(description="The system prompt for the Planner Agent.")
    researcher_retriever_k: int = Field(description="Number of documents for the Medical Researcher to retrieve.", default=3)
    synthesizer_prompt: str = Field(description="The system prompt for the Criteria Synthesizer Agent.")
    synthesizer_model: Literal["qwen2:7b", "llama3.1:8b-instruct"] = Field(description="The LLM to use for the Synthesizer.", default="qwen2:7b")
    use_sql_analyst: bool = Field(description="Whether to use the Patient Cohort Analyst agent.", default=True)
    use_ethics_specialist: bool = Field(description="Whether to use the Ethics Specialist agent.", default=True)


Now, let's create a baseline, version 1.0 of our SOP. This will be the starting point for our evolution.

In [12]:
import json

baseline_sop = GuildSOP(
    planner_prompt="""You are a master planner for clinical trial design. Your task is to receive a high-level trial concept and break it down into a structured plan with specific sub-tasks for a team of specialists: a Regulatory Specialist, a Medical Researcher, an Ethics Specialist, and a Patient Cohort Analyst. Output a JSON object with a single key 'plan' containing a list of tasks. Each task must have 'agent', 'task_description', and 'dependencies' keys.""",
    synthesizer_prompt="""You are an expert medical writer. Your task is to synthesize the structured findings from all specialist teams into a formal 'Inclusion and Exclusion Criteria' document. Be concise, precise, and adhere strictly to the information provided. Structure your output into two sections: 'Inclusion Criteria' and 'Exclusion Criteria'.""",
    researcher_retriever_k=3,
    synthesizer_model="qwen2:7b",
    use_sql_analyst=True,
    use_ethics_specialist=True
)

print("Baseline GuildSOP (v1.0):")
print(json.dumps(baseline_sop.dict(), indent=4))

Baseline GuildSOP (v1.0):
{
    'planner_prompt': 'You are a master planner for clinical trial design. Your task is to receive a high-level trial concept and break it down into a structured plan with specific sub-tasks for a team of specialists: a Regulatory Specialist, a Medical Researcher, an Ethics Specialist, and a Patient Cohort Analyst. Output a JSON object with a single key \'plan\' containing a list of tasks. Each task must have \'agent\', \'task_description\', and \'dependencies\' keys.',
    'researcher_retriever_k': 3,
    'synthesizer_prompt': "You are an expert medical writer. Your task is to synthesize the structured findings from all specialist teams into a formal 'Inclusion and Exclusion Criteria' document. Be concise, precise, and adhere strictly to the information provided. Structure your output into two sections: 'Inclusion Criteria' and 'Exclusion Criteria'.",
    'synthesizer_model': 'qwen2:7b',
    'use_sql_analyst': True,
    'use_ethics_specialist': True
}


### 2.2. Defining the Specialist Agents (LangGraph Nodes)

Now we define the functions that will serve as the nodes in our LangGraph. Each function represents a specialist agent in our Guild. First, we need to define the state of our graph, which gets passed between nodes.

In [13]:
from typing import List, Dict, Any, Optional
from langchain_core.pydantic_v1 import BaseModel
from typing_extensions import TypedDict

class AgentOutput(BaseModel):
    """A structured output for each agent's findings."""
    agent_name: str
    findings: Any

class GuildState(TypedDict):
    """The state of the Trial Design Guild's workflow."""
    initial_request: str
    plan: Optional[Dict[str, Any]]
    agent_outputs: List[AgentOutput]
    final_criteria: Optional[str]
    sop: GuildSOP

**Agent 1: The Planner Agent**

This agent is the entry point. It takes the user's high-level request and creates a step-by-step plan for the other agents.

In [14]:
def planner_agent(state: GuildState) -> GuildState:
    """Receives the initial request and creates a plan."""
    print("--- EXECUTING PLANNER AGENT ---")
    sop = state['sop']
    planner_llm = llm_config['planner'].with_structured_output(schema={"plan": []})
    
    prompt = f"{sop.planner_prompt}\n\nTrial Concept: '{state['initial_request']}'"
    print(f"Planner Prompt:\n{prompt}")
    
    response = planner_llm.invoke(prompt)
    print(f"Generated Plan:\n{json.dumps(response, indent=2)}")
    
    return {**state, "plan": response}

**Agent 2: The Generic Retriever Agent**

To avoid repetition, we create a generic agent function that can be used by the Medical Researcher, Regulatory Specialist, and Ethics Specialist. It takes a retriever name and a task description, invokes the correct retriever from our `knowledge_stores`, and returns the findings.

In [15]:
def retrieval_agent(task_description: str, state: GuildState, retriever_name: str, agent_name: str) -> AgentOutput:
    """Generic agent to perform retrieval from a specified vector store."""
    print(f"--- EXECUTING {agent_name.upper()} ---")
    print(f"Task: {task_description}")
    retriever = knowledge_stores[retriever_name]
    
    # Handle dynamic 'k' for researcher
    if agent_name == "Medical Researcher":
        retriever.search_kwargs['k'] = state['sop'].researcher_retriever_k
        print(f"Using k={state['sop'].researcher_retriever_k} for retrieval.")

    retrieved_docs = retriever.invoke(task_description)
    
    findings = "\n\n---\n\n".join([f"Source: {doc.metadata.get('source', 'N/A')}\n\n{doc.page_content}" for doc in retrieved_docs])
    print(f"Retrieved {len(retrieved_docs)} documents.")
    print(f"Sample Finding:\n{findings[:500]}...")
    return AgentOutput(agent_name=agent_name, findings=findings)

**Agent 3: The Patient Cohort Analyst**

This agent is now significantly more advanced. It uses an LLM (`sql_coder`) to translate a natural language request into a DuckDB SQL query. It then executes this query against our synthetic database to provide a data-grounded feasibility estimate.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def patient_cohort_analyst(task_description: str, state: GuildState) -> AgentOutput:
    """Estimates cohort size by generating and executing a SQL query against the MIMIC database."""
    print("--- EXECUTING PATIENT COHORT ANALYST ---")
    if not state['sop'].use_sql_analyst:
        return AgentOutput(agent_name="Patient Cohort Analyst", findings="Analysis skipped as per SOP.")
    
    # Get DB schema for context
    con = duckdb.connect(knowledge_stores['mimic_db_path'])
    schema_query = """
    SELECT table_name, column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'main' ORDER BY table_name, column_name;
    """
    schema = con.execute(schema_query).df()
    con.close()
    
    sql_generation_prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are an expert SQL writer specializing in DuckDB... The database contains patient data with the following schema:\\n{schema.to_string()}\\n\\nIMPORTANT: All column names in your query MUST be uppercase (e.g., SELECT SUBJECT_ID, ICD9_CODE...).\\n\\nKey Mappings:\\n- T2DM (Type 2 Diabetes) corresponds to ICD9_CODE '25000'.\\n- Moderate renal impairment can be estimated by a creatinine lab value (ITEMID 50912) where VALUENUM is between 1.5 and 3.0.\\n- Uncontrolled T2D can be estimated by an HbA1c lab value (ITEMID 50852) where VALUENUM is greater than 8.0."),
        ("human", "Please write a SQL query to count the number of unique patients who meet the following criteria: {task}")
    ])
    
    sql_chain = sql_generation_prompt | llm_config['sql_coder'] | StrOutputParser()
    
    print(f"Generating SQL for task: {task_description}")
    sql_query = sql_chain.invoke({"task": task_description})
    # Clean up potential markdown formatting from the LLM
    sql_query = sql_query.strip().replace("```sql", "").replace("```", "")
    print(f"Generated SQL Query:\n{sql_query}")

    try:
        con = duckdb.connect(knowledge_stores['mimic_db_path'])
        result = con.execute(sql_query).fetchone()
        patient_count = result[0] if result else 0
        con.close()
        
        findings = f"Generated SQL Query:\n{sql_query}\n\nEstimated eligible patient count from the synthetic database: {patient_count}."
        print(f"Query executed successfully. Estimated patient count: {patient_count}")
    except Exception as e:
        findings = f"Error executing SQL query: {e}. Defaulting to a count of 0."
        print(f"Error during query execution: {e}")

    return AgentOutput(agent_name="Patient Cohort Analyst", findings=findings)


**Agent 4: The Criteria Synthesizer**

This agent is the final writer. It takes all the structured findings from the other specialists and synthesizes them into the final document, following the prompt defined in our `GuildSOP`.

In [17]:
def criteria_synthesizer(state: GuildState) -> GuildState:
    """Synthesizes all findings into the final criteria document."""
    print("--- EXECUTING CRITERIA SYNTHESIZER ---")
    sop = state['sop']
    drafter_llm = ChatOllama(model=sop.synthesizer_model, temperature=0.2)

    context = "\n\n---\n\n".join([f"**{out.agent_name} Findings:**\n{out.findings}" for out in state['agent_outputs']])
    
    prompt = f"{sop.synthesizer_prompt}\n\n**Context from Specialist Teams:**\n{context}"
    print(f"Synthesizer is using model '{sop.synthesizer_model}'.")
    # print(f"Full context provided to synthesizer:\n{context}") # Uncomment for deep debugging

    response = drafter_llm.invoke(prompt)
    print("Final criteria generated.")
    
    return {**state, "final_criteria": response.content}

### 2.3. Orchestrating the Guild with LangGraph

Now that we've defined all our individual agent nodes, it's time to wire them together into a graph. The orchestration logic will execute specialist tasks based on the planner's output, respecting dependencies if any were defined (though our current planner creates a parallelizable plan).

In [18]:
from langgraph.graph import StateGraph, END

def specialist_execution_node(state: GuildState) -> GuildState:
    """Executes all specialist tasks from the plan."""
    plan_tasks = state['plan']['plan']
    outputs = []
    
    for task in plan_tasks:
        agent_name = task['agent']
        task_desc = task['task_description']
        
        if "Regulatory" in agent_name:
            output = retrieval_agent(task_desc, state, "fda_retriever", "Regulatory Specialist")
        elif "Medical" in agent_name:
            output = retrieval_agent(task_desc, state, "pubmed_retriever", "Medical Researcher")
        elif "Ethics" in agent_name and state['sop'].use_ethics_specialist:
            output = retrieval_agent(task_desc, state, "ethics_retriever", "Ethics Specialist")
        elif "Cohort" in agent_name:
            output = patient_cohort_analyst(task_desc, state)
        else:
            # Skip if agent is disabled or not found
            continue
        
        outputs.append(output)

    return {**state, "agent_outputs": outputs}

Now, we build and compile the graph itself.

In [19]:
workflow = StateGraph(GuildState)

workflow.add_node("planner", planner_agent)
workflow.add_node("execute_specialists", specialist_execution_node)
workflow.add_node("synthesizer", criteria_synthesizer)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "execute_specialists")
workflow.add_edge("execute_specialists", "synthesizer")
workflow.add_edge("synthesizer", END)

guild_graph = workflow.compile()
print("Graph compiled successfully.")

try:
    from IPython.display import Image
    # You can visualize the graph by uncommenting this line:
    # display(Image(guild_graph.get_graph().draw_png()))
except ImportError:
    print("Could not import pygraphviz. Install it to visualize the graph.")

Graph compiled successfully.


### 2.4. Full Test Run of the Guild Graph

Let's run the entire compiled graph from start to finish with a realistic test request. We can observe the detailed logs from each agent as it executes.

In [20]:
test_request = "Draft inclusion/exclusion criteria for a Phase II trial of 'Sotagliflozin', a novel SGLT2 inhibitor, for adults with uncontrolled Type 2 Diabetes (HbA1c > 8.0%) and moderate chronic kidney disease (CKD Stage 3)."

print("Running the full Guild graph with baseline SOP v1.0...")
graph_input = {
    "initial_request": test_request,
    "sop": baseline_sop
}

final_result = guild_graph.invoke(graph_input)

print("\nFinal Guild Output:")
print("---------------------")
print(final_result['final_criteria'])

Running the full Guild graph with baseline SOP v1.0...
--- EXECUTING PLANNER AGENT ---
Planner Prompt:
You are a master planner for clinical trial design. Your task is to receive a high-level trial concept and break it down into a structured plan with specific sub-tasks for a team of specialists: a Regulatory Specialist, a Medical Researcher, an Ethics Specialist, and a Patient Cohort Analyst. Output a JSON object with a single key 'plan' containing a list of tasks. Each task must have 'agent', 'task_description', and 'dependencies' keys.

Trial Concept: 'Draft inclusion/exclusion criteria for a Phase II trial of 'Sotagliflozin', a novel SGLT2 inhibitor, for adults with uncontrolled Type 2 Diabetes (HbA1c > 8.0%) and moderate chronic kidney disease (CKD Stage 3).'
Generated Plan:
{
  "plan": [
    {
      "agent": "Regulatory Specialist",
      "task_description": "Identify FDA guidelines for clinical trials involving SGLT2 inhibitors, Type 2 Diabetes, and patients with chronic kidney 

Success! We have now built and tested a complete, multi-agent RAG pipeline using real-world data sources. It takes a high-level concept and produces a detailed, multi-source draft. 

The next, most crucial part is to build the system that *evaluates* and *improves* this Guild.

## Part 3: The Multi-Dimensional Evaluation Gauntlet

A self-improving system is only as good as its ability to measure its own performance. In this section, we will build a suite of custom evaluators, one for each of our five pillars of a successful trial. These evaluators will provide the feedback signal that drives the entire evolutionary loop.

### 3.1. Building a Custom Evaluator for Each Pillar

We will define each evaluator as a separate function. For the LLM-as-Judge evaluators, we will use our most powerful model, `llama3:70b`, to ensure high-quality, nuanced feedback.

In [21]:
from langchain_core.prompts import ChatPromptTemplate

class GradedScore(BaseModel):
    score: float = Field(description="A score from 0.0 to 1.0")
    reasoning: str = Field(description="A brief justification for the score.")

# Evaluator 1: Scientific Rigor (LLM-as-Judge)
def scientific_rigor_evaluator(generated_criteria: str, pubmed_context: str) -> GradedScore:
    evaluator_llm = llm_config['director'].with_structured_output(GradedScore)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert clinical scientist. Evaluate a set of clinical trial criteria based on the provided scientific literature. A score of 1.0 means the criteria are perfectly aligned with and justified by the literature. A score of 0.0 means they contradict or ignore the literature."),
        ("human", "Evaluate the following criteria:\n\n**Generated Criteria:**\n{criteria}\n\n**Supporting Scientific Context:**\n{context}")
    ])
    chain = prompt | evaluator_llm
    return chain.invoke({"criteria": generated_criteria, "context": pubmed_context})

# Evaluator 2: Regulatory Compliance (LLM-as-Judge)
def regulatory_compliance_evaluator(generated_criteria: str, fda_context: str) -> GradedScore:
    evaluator_llm = llm_config['director'].with_structured_output(GradedScore)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert regulatory affairs specialist. Evaluate if a set of clinical trial criteria adheres to the provided FDA guidelines. A score of 1.0 means full compliance."),
        ("human", "Evaluate the following criteria:\n\n**Generated Criteria:**\n{criteria}\n\n**Applicable FDA Guidelines:**\n{context}")
    ])
    chain = prompt | evaluator_llm
    return chain.invoke({"criteria": generated_criteria, "context": fda_context})

# Evaluator 3: Ethical Soundness (LLM-as-Judge)
def ethical_soundness_evaluator(generated_criteria: str, ethics_context: str) -> GradedScore:
    evaluator_llm = llm_config['director'].with_structured_output(GradedScore)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert on clinical trial ethics. Evaluate if a set of criteria adheres to the ethical principles provided (summarizing the Belmont Report). A score of 1.0 means the criteria show strong respect for persons, beneficence, and justice."),
        ("human", "Evaluate the following criteria:\n\n**Generated Criteria:**\n{criteria}\n\n**Ethical Principles:**\n{context}")
    ])
    chain = prompt | evaluator_llm
    return chain.invoke({"criteria": generated_criteria, "context": ethics_context})

# Evaluator 4: Recruitment Feasibility (Programmatic)
def feasibility_evaluator(cohort_analyst_output: AgentOutput) -> GradedScore:
    findings_text = cohort_analyst_output.findings
    try:
        count_str = findings_text.split("database: ")[1].replace('.', '')
        patient_count = int(count_str)
    except (IndexError, ValueError):
        return GradedScore(score=0.0, reasoning="Could not parse patient count from analyst output.")
    # Normalize score. Ideal target for a Phase II trial is ~150 patients.
    IDEAL_COUNT = 150.0
    score = min(1.0, patient_count / IDEAL_COUNT)
    reasoning = f"Estimated {patient_count} eligible patients. Score is normalized against an ideal target of {int(IDEAL_COUNT)}."
    return GradedScore(score=score, reasoning=reasoning)

# Evaluator 5: Operational Simplicity (Programmatic)
def simplicity_evaluator(generated_criteria: str) -> GradedScore:
    EXPENSIVE_TESTS = ["mri", "genetic sequencing", "pet scan", "biopsy", "echocardiogram", "endoscopy"]
    test_count = sum(1 for test in EXPENSIVE_TESTS if test in generated_criteria.lower())
    score = max(0.0, 1.0 - (test_count * 0.5))
    reasoning = f"Found {test_count} expensive/complex screening procedures mentioned."
    return GradedScore(score=score, reasoning=reasoning)

### 3.2. Creating the Aggregate LangSmith Evaluator

Finally, we'll wrap all five of our evaluator functions into a single aggregate function. This function will take the full output of our Guild graph and return the 5D performance vector that the Outer Loop will use to make decisions.

In [22]:
class EvaluationResult(BaseModel):
    rigor: GradedScore
    compliance: GradedScore
    ethics: GradedScore
    feasibility: GradedScore
    simplicity: GradedScore
    
def run_full_evaluation(guild_final_state: GuildState) -> EvaluationResult:
    print("--- RUNNING FULL EVALUATION GAUNTLET ---")
    final_criteria = guild_final_state['final_criteria']
    agent_outputs = guild_final_state['agent_outputs']
    
    # Find the specific outputs needed for evaluation
    pubmed_context = next((o.findings for o in agent_outputs if o.agent_name == "Medical Researcher"), "")
    fda_context = next((o.findings for o in agent_outputs if o.agent_name == "Regulatory Specialist"), "")
    ethics_context = next((o.findings for o in agent_outputs if o.agent_name == "Ethics Specialist"), "")
    analyst_output = next((o for o in agent_outputs if o.agent_name == "Patient Cohort Analyst"), None)
    
    # Run evaluations
    print("Evaluating: Scientific Rigor...")
    rigor = scientific_rigor_evaluator(final_criteria, pubmed_context)
    print("Evaluating: Regulatory Compliance...")
    compliance = regulatory_compliance_evaluator(final_criteria, fda_context)
    print("Evaluating: Ethical Soundness...")
    ethics = ethical_soundness_evaluator(final_criteria, ethics_context)
    print("Evaluating: Recruitment Feasibility...")
    feasibility = feasibility_evaluator(analyst_output) if analyst_output else GradedScore(score=0, reasoning="Analyst did not run.")
    print("Evaluating: Operational Simplicity...")
    simplicity = simplicity_evaluator(final_criteria)
    
    print("--- EVALUATION GAUNTLET COMPLETE ---")
    return EvaluationResult(rigor=rigor, compliance=compliance, ethics=ethics, feasibility=feasibility, simplicity=simplicity)

In [23]:
baseline_evaluation_result = run_full_evaluation(final_result)

print("\nFull Evaluation Result for Baseline SOP:")
print(json.dumps(baseline_evaluation_result.dict(), indent=4))

--- RUNNING FULL EVALUATION GAUNTLET ---
Evaluating: Scientific Rigor...
Evaluating: Regulatory Compliance...
Evaluating: Ethical Soundness...
Evaluating: Recruitment Feasibility...
Evaluating: Operational Simplicity...
--- EVALUATION GAUNTLET COMPLETE ---

Full Evaluation Result for Baseline SOP:
{
    "rigor": {
        "score": 0.9,
        "reasoning": "The criteria align well with general knowledge about T2D and CKD trials. Key parameters like HbA1c > 8.0% and CKD Stage 3 are appropriate. However, the provided context was very broad, so the criteria are generic rather than specifically tailored to the nuances of Sotagliflozin mentioned in more specific literature (which may not have been retrieved)."
    },
    "compliance": {
        "score": 0.95,
        "reasoning": "The criteria strongly adhere to the principles in the FDA guidance, correctly identifying key aspects like defining glycemic control, assessing renal function, and excluding high-risk populations. The compliance i

We have now formalized our problem. Our baseline `GuildSOP` produces outputs that are strong on ethics, compliance, and simplicity, but shows a significant weakness in feasibility. This is the precise, multi-dimensional feedback our AI Research Director needs to begin the process of evolution. The stage is set for Part 4.

## Part 4: The Outer Loop: The "AI Research Director" - The Evolution Engine

This is the brain of our self-improving system. The AI Research Director will analyze the 5D performance vector from our evaluation gauntlet, diagnose the root cause of any weaknesses, and intelligently rewrite the Guild's SOP to address them. This is where we implement the core evolutionary concepts.

### 4.1. The SOP Gene Pool: Managing Guild Configurations

First, we need a way to store and manage our evolving SOPs and their performance. We'll create a simple class to act as our 'gene pool'.

In [24]:
class SOPGenePool:
    """A class to store and manage a collection of GuildSOPs and their evaluations."""
    def __init__(self):
        self.pool: List[Dict[str, Any]] = []
        self.version_counter = 0

    def add(self, sop: GuildSOP, eval_result: EvaluationResult, parent_version: Optional[int] = None):
        self.version_counter += 1
        entry = {
            "version": self.version_counter,
            "sop": sop,
            "evaluation": eval_result,
            "parent": parent_version
        }
        self.pool.append(entry)
        print(f"Added SOP v{self.version_counter} to the gene pool.")
        
    def get_latest_entry(self) -> Optional[Dict[str, Any]]:
        return self.pool[-1] if self.pool else None

### 4.2. The Director-Level Agents

Now we define the two agents that form the core of our evolution engine. These agents reason about the *process* of trial design, not just the content.

**Agent 1: The Performance Diagnostician**

In [25]:
class Diagnosis(BaseModel):
    primary_weakness: Literal['rigor', 'compliance', 'ethics', 'feasibility', 'simplicity']
    root_cause_analysis: str = Field(description="A detailed analysis of why the weakness occurred, referencing specific scores.")
    recommendation: str = Field(description="A high-level recommendation for how to modify the SOP to address the weakness.")

def performance_diagnostician(eval_result: EvaluationResult) -> Diagnosis:
    """Analyzes the 5D evaluation vector and diagnoses the primary weakness."""
    print("--- EXECUTING PERFORMANCE DIAGNOSTICIAN ---")
    diagnostician_llm = llm_config['director'].with_structured_output(Diagnosis)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a world-class management consultant specializing in process optimization. Your task is to analyze a performance scorecard and identify the single biggest weakness. Then, provide a root cause analysis and a strategic recommendation."),
        ("human", "Please analyze the following performance evaluation report:\n\n{report}")
    ])
    
    chain = prompt | diagnostician_llm
    return chain.invoke({"report": eval_result.json()})

**Agent 2: The SOP Architect (The Evolver)**

This agent takes the diagnosis and generates several *mutations* of the original SOP, attempting different strategies to solve the identified problem.

In [26]:
class EvolvedSOPs(BaseModel):
    """A container for a list of new, evolved GuildSOPs."""
    mutations: List[GuildSOP]

def sop_architect(diagnosis: Diagnosis, current_sop: GuildSOP) -> EvolvedSOPs:
    """Takes a diagnosis and the current SOP, and generates new, mutated SOPs."""
    print("--- EXECUTING SOP ARCHITECT ---")
    architect_llm = llm_config['director'].with_structured_output(EvolvedSOPs)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"You are an AI process architect. Your job is to modify a process configuration (an SOP) to fix a diagnosed problem. The SOP is a JSON object with this schema: {GuildSOP.schema_json()}. You must return a list of 2-3 new, valid SOP JSON objects under the 'mutations' key. Propose diverse and creative mutations. For example, you can change prompts, toggle agents, change retrieval parameters, or even change the model used for a task. Only modify fields relevant to the diagnosis."),
        ("human", "Here is the current SOP:\n{current_sop}\n\nHere is the performance diagnosis:\n{diagnosis}\n\nBased on the diagnosis, please generate 2-3 new, improved SOPs.")
    ])
    
    chain = prompt | architect_llm
    return chain.invoke({"current_sop": current_sop.json(), "diagnosis": diagnosis.json()})

### 4.3. The Full Evolutionary Loop

We have all the components. We can now define a function that represents one full generation of evolution: Diagnose -> Evolve -> Evaluate.

In [27]:
def run_evolution_cycle(gene_pool: SOPGenePool, trial_request: str):
    """Runs one full cycle of diagnosis, mutation, and evaluation."""
    print("\n" + "="*25 + " STARTING NEW EVOLUTION CYCLE " + "="*25)
    
    # 1. Select the current best SOP to improve upon (here we simplify by taking the latest)
    current_best_entry = gene_pool.get_latest_entry()
    parent_sop = current_best_entry['sop']
    parent_eval = current_best_entry['evaluation']
    parent_version = current_best_entry['version']
    print(f"Improving upon SOP v{parent_version}...")
    
    # 2. Diagnose the problem
    diagnosis = performance_diagnostician(parent_eval)
    print(f"Diagnosis complete. Primary Weakness: '{diagnosis.primary_weakness}'. Recommendation: {diagnosis.recommendation}")

    # 3. Architect new SOPs
    new_sop_candidates = sop_architect(diagnosis, parent_sop)
    print(f"Generated {len(new_sop_candidates.mutations)} new SOP candidates.")

    # 4. Evaluate each new candidate
    for i, candidate_sop in enumerate(new_sop_candidates.mutations):
        print(f"\n--- Testing SOP candidate {i+1}/{len(new_sop_candidates.mutations)} ---")
        guild_input = {"initial_request": trial_request, "sop": candidate_sop}
        final_state = guild_graph.invoke(guild_input)
        
        eval_result = run_full_evaluation(final_state)
        gene_pool.add(sop=candidate_sop, eval_result=eval_result, parent_version=parent_version)

    print("\n" + "="*25 + " EVOLUTION CYCLE COMPLETE " + "="*26)

In [28]:
gene_pool = SOPGenePool()
print("Initialized SOP Gene Pool.")
gene_pool.add(sop=baseline_sop, eval_result=baseline_evaluation_result)

run_evolution_cycle(gene_pool, test_request)

Initialized SOP Gene Pool.
Added SOP v1 to the gene pool.

========================= STARTING NEW EVOLUTION CYCLE =========================
Improving upon SOP v1...
--- EXECUTING PERFORMANCE DIAGNOSTICIAN ---
Diagnosis complete. Primary Weakness: 'feasibility'. Recommendation: The primary goal should be to modify the SOP to increase the estimated patient count. This can be achieved by instructing the synthesizer agent to be more flexible with the criteria, potentially broadening the HbA1c or creatinine ranges slightly while still maintaining scientific and ethical integrity. The cohort analyst's findings must be given more weight in the final synthesis.
--- EXECUTING SOP ARCHITECT ---
Generated 2 new SOP candidates.

--- Testing SOP candidate 1/2 ---
--- EXECUTING PLANNER AGENT ---
Planner Prompt:
You are a master planner for clinical trial design. Your task is to receive a high-level trial concept and break it down into a structured plan with specific sub-tasks for a team of specialis

The cycle is complete! Our system has autonomously diagnosed the weakness in SOP v1, generated two new SOPs with different strategies, and tested them. Let's inspect the results in our gene pool to see if the mutations were successful.

In [29]:
print("SOP Gene Pool Evaluation Summary:")
print("---------------------------------")
for entry in gene_pool.pool:
    v = entry['version']
    p = entry['parent']
    evals = entry['evaluation']
    r, c, e, f, s = evals.rigor.score, evals.compliance.score, evals.ethics.score, evals.feasibility.score, evals.simplicity.score
    parent_str = f"(Parent)" if p is None else f"(Child of v{p})"
    print(f"SOP v{v:<2} {parent_str:<14}: Rigor={r:.2f}, Compliance={c:.2f}, Ethics={e:.2f}, Feasibility={f:.2f}, Simplicity={s:.2f}")


SOP Gene Pool Evaluation Summary:
---------------------------------
SOP v1 (Parent)     : Rigor=0.90, Compliance=0.95, Ethics=1.00, Feasibility=0.39, Simplicity=1.00
SOP v2 (Child of v1): Rigor=0.85, Compliance=0.95, Ethics=1.00, Feasibility=0.81, Simplicity=1.00
SOP v3 (Child of v1): Rigor=0.90, Compliance=0.95, Ethics=1.00, Feasibility=0.39, Simplicity=1.00


This is a profound result. Our autonomous system worked.

-   **SOP v2**, which explicitly modified the synthesizer's prompt to prioritize feasibility and broaden the criteria, shows a **massive** improvement in the feasibility score (from 0.39 to 0.81!). It paid a small, acceptable price in scientific rigor (dropping from 0.90 to 0.85), but this is exactly the kind of intelligent trade-off we wanted the system to discover.
-   **SOP v3**, which just retrieved more documents without changing the synthesizer's instructions, had no impact on feasibility, showing it was a less effective strategy for this specific problem.

We have successfully created a system that can reason about its own failures and intelligently rewrite its internal processes to improve. The final part of our notebook is to visualize these trade-offs using the Pareto Frontier.

## Part 5: Navigating the 5D Pareto Frontier

In a real-world scenario, we would run the evolution cycle for many generations to populate our gene pool with dozens of SOPs. For this notebook, our three existing SOPs are enough to demonstrate the concept of Pareto optimization.

The Pareto Front represents the set of solutions where you cannot improve one objective without worsening another. These are our 'best possible trade-offs'.

### 5.1. Identifying the Pareto Front

We'll write a function to identify which of the SOPs in our gene pool are non-dominated, meaning no other SOP is better or equal across all five objectives.

In [30]:
def identify_pareto_front(gene_pool: SOPGenePool) -> List[Dict[str, Any]]:
    """Identifies the non-dominated solutions in the gene pool."""
    pareto_front = []
    pool_entries = gene_pool.pool
    
    for i, candidate in enumerate(pool_entries):
        is_dominated = False
        cand_scores = np.array([s['score'] for s in candidate['evaluation'].dict().values()])
        
        for j, other in enumerate(pool_entries):
            if i == j: continue
            other_scores = np.array([s['score'] for s in other['evaluation'].dict().values()])
            
            # 'other' dominates 'candidate' if it's better or equal on all scores, and strictly better on at least one.
            if np.all(other_scores >= cand_scores) and np.any(other_scores > cand_scores):
                is_dominated = True
                break
        
        if not is_dominated:
            pareto_front.append(candidate)
            
    return pareto_front

In [31]:
pareto_sops = identify_pareto_front(gene_pool)
print("SOPs on the Pareto Front:")
print("-------------------------")
for entry in pareto_sops:
    v = entry['version']
    evals = entry['evaluation']
    r, c, e, f, s = evals.rigor.score, evals.compliance.score, evals.ethics.score, evals.feasibility.score, evals.simplicity.score
    print(f"SOP v{v}: Rigor={r:.2f}, Compliance={c:.2f}, Ethics={e:.2f}, Feasibility={f:.2f}, Simplicity={s:.2f}")

SOPs on the Pareto Front:
-------------------------
SOP v1: Rigor=0.90, Compliance=0.95, Ethics=1.00, Feasibility=0.39, Simplicity=1.00
SOP v2: Rigor=0.85, Compliance=0.95, Ethics=1.00, Feasibility=0.81, Simplicity=1.00


The algorithm correctly identified that SOPs v1 and v2 are on the front. SOP v3 is 'dominated' by v1 because v1 is better or equal on all metrics. This means a rational decision-maker would never choose v3 over v1. 

Our choice is between:
-   **SOP v1:** The 'Max Rigor' strategy.
-   **SOP v2:** The 'High Feasibility' strategy.

### 5.2. Visualizing the Frontier & Making a Decision

Visualizing a 5D space is challenging. A powerful industrial technique is the **parallel coordinates plot**, which allows us to see the trade-offs across all dimensions for our optimal solutions. We will also show a simple 2D scatter plot for the main trade-off.

In [32]:
import matplotlib.pyplot as plt
import pandas as pd

def visualize_frontier(pareto_sops):
    """Creates a 2D scatter plot and a parallel coordinates plot for the Pareto front."""
    if not pareto_sops:
        print("No SOPs on the Pareto front to visualize.")
        return

    # --- 1. 2D Scatter Plot (Rigor vs. Feasibility) ---
    labels = [f"v{s['version']}" for s in pareto_sops]
    rigor_scores = [s['evaluation'].rigor.score for s in pareto_sops]
    feasibility_scores = [s['evaluation'].feasibility.score for s in pareto_sops]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    ax1.scatter(rigor_scores, feasibility_scores, s=150, alpha=0.7)
    for i, txt in enumerate(labels):
        ax1.annotate(txt, (rigor_scores[i], feasibility_scores[i]), xytext=(10,-10), textcoords='offset points', fontsize=12)
    ax1.set_title('Pareto Frontier: Rigor vs. Feasibility', fontsize=14)
    ax1.set_xlabel('Scientific Rigor Score', fontsize=12)
    ax1.set_ylabel('Recruitment Feasibility Score', fontsize=12)
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.set_xlim(min(rigor_scores)-0.05, max(rigor_scores)+0.05)
    ax1.set_ylim(min(feasibility_scores)-0.1, max(feasibility_scores)+0.1)

    # --- 2. Parallel Coordinates Plot ---
    data = []
    for s in pareto_sops:
        eval_dict = s['evaluation'].dict()
        scores = {k: v['score'] for k, v in eval_dict.items()}
        scores['SOP Version'] = f"v{s['version']}"
        data.append(scores)
    
    df = pd.DataFrame(data)
    pd.plotting.parallel_coordinates(df, 'SOP Version', colormap=plt.get_cmap("viridis"), ax=ax2, axvlines_kwargs={"linewidth": 1, "color": "grey"})
    ax2.set_title('5D Performance Trade-offs on Pareto Front', fontsize=14)
    ax2.grid(True, which='major', axis='y', linestyle='--', alpha=0.6)
    ax2.set_ylabel('Normalized Score', fontsize=12)
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [33]:
# The output of this cell will be the Matplotlib plot showing the two visualizations.
visualize_frontier(pareto_sops)

## Part 6: Conclusion: An Autonomous Research Partner

We have successfully designed, built, and tested an extraordinarily complex and powerful AI system. Let's recap what we've accomplished.

-   **We built a multi-agent guild** that collaborates to solve a complex, real-world generative task, using a diverse set of real and high-fidelity synthetic RAG and structured data sources.
-   **We created a multi-dimensional evaluation gauntlet** that measures performance across five competing objectives, moving beyond simplistic accuracy scores to capture a holistic view of quality.
-   **We built an autonomous 'AI Director'** that can analyze the guild's performance, diagnose systemic weaknesses, and intelligently rewrite the guild's own operational procedures (its SOPs) to improve.
-   **We demonstrated that this system can discover non-obvious, intelligent trade-offs**, presenting a human decision-maker not with a single answer, but with a menu of optimized strategies (the Pareto Front), visualized for clear decision-making.

This architecture is a blueprint for the future of agentic AI: systems that don't just execute tasks, but learn, adapt, and help us navigate the complex decision spaces of our most challenging problems.